In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import depthcharge as dc
import numpy as np
from depthcharge.encoders import PeakEncoder
from depthcharge.data import SpectrumDataset, StreamingSpectrumDataset

/Users/dalfr76/Documents/mass_spec_classifier/dc_env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


This is the ideal way to read in spectra for the problem we are trying to solve, but I'll see if I can get it to work.

In [3]:
import os

orbi_lumos_directory = 'orbitrap_lumos'
#orbi_lumos_mzmls = [os.path.join(orbi_lumos_directory, file) 
#             for file in os.listdir(orbi_lumos_directory) 
#             if os.path.isfile(os.path.join(orbi_lumos_directory, file))]
orbi_lumos_mzmls = ["orbitrap_lumos/08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML",
                    "orbitrap_lumos/BD20180604_PTRC_DP1-Microscale_SS_Plex3_Proteome_F20.mzML"]
q_exactive_hfx_directory = 'Q_exactive_hfx'
#q_exactive_hfx_mzmls = [os.path.join(q_exactive_hfx_directory, file) 
#             for file in os.listdir(q_exactive_hfx_directory) 
#             if os.path.isfile(os.path.join(q_exactive_hfx_directory, file))]
q_exactive_hfx_mzmls = ["Q_exactive_hfx/19CPTAC_LUAD_W_BI_20180730_KL_f08.mzML",
                        "Q_exactive_hfx/KY20201205_TV_CarboDocPDX_Proteome_f07.mzML"]
# Assign labels for each list
ms_labeler = {}

# Add files from list1 with label 0
for filename in orbi_lumos_mzmls:
    ms_labeler[filename] = 0

# Add files from list2 with label 1
for filename in q_exactive_hfx_mzmls:
    ms_labeler[filename] = 1

all_mzmls = orbi_lumos_mzmls + q_exactive_hfx_mzmls

label_decoder = {
    0: orbi_lumos_directory,
    1: q_exactive_hfx_directory,
}

#orbi_lumos_mzmls = './orbitrap_lumos/08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML'
processing_fn = [
    #https://spectrum-utils.readthedocs.io/en/latest/api.html#spectrum_utils.spectrum.MsmsSpectrum.set_mz_range
    dc.data.preprocessing.set_mz_range(min_mz=0),
    #https://spectrum-utils.readthedocs.io/en/latest/api.html#spectrum_utils.spectrum.MsmsSpectrum.filter_intensity
    dc.data.preprocessing.filter_intensity(min_intensity = 0.1),  # Change this value to allow number of peaks per spectrum
    #https://spectrum-utils.readthedocs.io/en/latest/api.html#spectrum_utils.spectrum.MsmsSpectrum.scale_intensity
    dc.data.preprocessing.scale_intensity(scaling=None),  # Might want to change later
    dc.data.preprocessing.scale_to_unit_norm,  # Not sure what this is yet
]

all_data = list()
all_labels = list()
for mzml_file in all_mzmls:
    # Preprocess the mzML file
    df = dc.data.spectra_to_df(
        mzml_file,
        progress=True,
        preprocessing_fn=processing_fn
    )
    
    # Encode peaks
    encoder = PeakEncoder(100)
    dataset = StreamingSpectrumDataset(df, batch_size=54852)
    embeddings = None

    for spectrum in dataset:
        mz_values = spectrum["mz_array"]
        intensities = spectrum["intensity_array"]
        stack = torch.stack((mz_values, intensities), dim=2)
        embedded_batch = encoder.forward(stack)
        embeddings = embedded_batch.detach()
        
        # Append the data and labels
        all_data.append(embeddings)
        all_labels.extend([ms_labeler[mzml_file]] * embeddings.size(0))  # Repeat the label for all spectra
'''  
df = dc.data.spectra_to_df(
    mzml,
    progress=True,
    preprocessing_fn=processing_fn
)
encoder = PeakEncoder(100)
dataset = SpectrumDataset(df, batch_size=54852)
'''

08CPTAC_C_GBM_W_PNNL_20210830_B2S3_f20.mzML: 100%|██████████| 54852/54852 [00:24<00:00, 2258.02 spectra/s]
BD20180604_PTRC_DP1-Microscale_SS_Plex3_Proteome_F20.mzML: 100%|██████████| 49246/49246 [00:18<00:00, 2658.64 spectra/s]
KY20201205_TV_CarboDocPDX_Proteome_f07.mzML: 100%|██████████| 48845/48845 [00:23<00:00, 2040.89 spectra/s]


'  \ndf = dc.data.spectra_to_df(\n    mzml,\n    progress=True,\n    preprocessing_fn=processing_fn\n)\nencoder = PeakEncoder(100)\ndataset = SpectrumDataset(df, batch_size=54852)\n'

Builds the tensors together in a memory friendly way

In [4]:
#for i in all_data:
#    print(i.shape)


def join_tensors(tensor1, tensor2):

  n, m, d = tensor1.shape
  x, z, d = tensor2.shape

  # Calculate padding needed
  padding_needed = abs(z - m)

  # Create a padding tensor filled with zeros
  

  # Pad tensor1 along the second dimension
  if(m > z): 
    padding = torch.zeros(x, padding_needed, d)
    padded_tensor = torch.cat((tensor2, padding), dim=1)
    result_tensor = torch.cat((tensor1, padded_tensor), dim=0)
  else:
    padding = torch.zeros(m, padding_needed, d)
    padded_tensor = torch.cat((tensor1, padding), dim=1)
    result_tensor = torch.cat((padded_tensor, tensor2), dim=0)
  # Concatenate the padded tensor1 and tensor2 along the first dimension
  

  return result_tensor

joined_embeddings_train = join_tensors(all_data[0], all_data[2])
joined_embeddings_test = join_tensors(all_data[1], all_data[3])



In [10]:
all_data[1][0]

tensor([[-0.0556,  0.2482, -0.1343,  ...,  0.4439, -0.1518,  0.4000],
        [ 0.3341,  0.4097, -0.2814,  ..., -0.1089,  0.8045, -0.4933],
        [ 0.3465,  0.5483,  0.0394,  ..., -0.4057,  0.5563, -0.9627],
        ...,
        [ 0.1386, -0.0148,  0.0277,  ..., -0.1126, -0.0243, -0.2350],
        [ 0.1386, -0.0148,  0.0277,  ..., -0.1126, -0.0243, -0.2350],
        [ 0.1386, -0.0148,  0.0277,  ..., -0.1126, -0.0243, -0.2350]])

In [11]:
joined_embeddings[39793]

tensor([[-0.0556,  0.2482, -0.1343,  ...,  0.4439, -0.1518,  0.4000],
        [ 0.3341,  0.4097, -0.2814,  ..., -0.1089,  0.8045, -0.4933],
        [ 0.3465,  0.5483,  0.0394,  ..., -0.4057,  0.5563, -0.9627],
        ...,
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000],
        [ 0.0000,  0.0000,  0.0000,  ...,  0.0000,  0.0000,  0.0000]])

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np


num_epochs = 3
# Stack all embeddings into a single tensor
all_data_tensor = joined_embeddings

# Convert labels to a PyTorch tensor
all_labels_tensor = torch.tensor(all_labels, dtype=torch.long)

# Create a TensorDataset and DataLoader
dataset = TensorDataset(all_data_tensor, all_labels_tensor)
dataloader = DataLoader(dataset, batch_size=64, shuffle=True)

# 2. Model Definition (CNN approach - recommended)
class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 32, kernel_size=(3, 3), padding=1)  # Input channels = 1
        self.relu1 = nn.ReLU()
        self.pool1 = nn.MaxPool2d(kernel_size=(2, 2))
        self.conv2 = nn.Conv2d(32, 64, kernel_size=(3, 3), padding=1)
        self.relu2 = nn.ReLU()
        self.pool2 = nn.MaxPool2d(kernel_size=(2, 2))
        self.flatten = nn.Flatten()
        self.fc = nn.Linear(64 * 50 * 27, 2) #nn.Linear(64 * 50 * 25, 2)  # Adjust this depending on the input

    def forward(self, x):
        # Ensure input has correct dimensions
        if x.dim() == 3:  # Handle input shape (N, H, W)
            x = x.unsqueeze(1)  # Add channel dimension
        x = self.pool1(self.relu1(self.conv1(x)))
        x = self.pool2(self.relu2(self.conv2(x)))
        x = self.flatten(x)
        x = self.fc(x)
        return x

# For Rob
if torch.backends.mps.is_available():
    device = torch.device("mps")
elif torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")
#device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
# For Nvidia GPU
#device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = Classifier().to(device)

# 3. Loss Function and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 4. Training loop with reshaping for CNN
for epoch in range(num_epochs):
    for inputs, labels in dataloader:
        inputs, labels = inputs.to(device), labels.to(device)

        # Reshape inputs for CNN
        if inputs.dim() == 3:
            inputs = inputs.unsqueeze(1)  # (N, H, W) -> (N, C, H, W)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# During evaluation, move inputs and labels to MPS (or CUDA)
model.eval()  # Set the model to evaluation mode
with torch.no_grad():
    correct = 0
    total = 0
    for inputs, labels in dataloader:
        # Move data to the same device as the model
        inputs, labels = inputs.to(device), labels.to(device)

        # Perform forward pass
        outputs = model(inputs)

        # Compute predictions and accuracy
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f'Accuracy: {100 * correct / total:.2f}%')

Epoch [1/3], Loss: 0.0000
Epoch [2/3], Loss: 0.0000
Epoch [3/3], Loss: 0.0000
Accuracy: 100.00%
